In [1]:
import os
import shutil
import numpy as np
from tqdm import tqdm

def split_dataset(
    input_dir,
    output_dir,
    defect_types,
    train_ratio=0.83,
    val_ratio=0.10,
    test_ratio=0.07,
    seed=42
):
    """
    Split dataset into train/val/test sets while preserving directory structure
    
    Args:
        input_dir: Path to original dataset (with images/ and masks/ folders)
        output_dir: Path to save split dataset
        defect_types: List of defect category folders
        train_ratio: Proportion of data for training
        val_ratio: Proportion of data for validation
        test_ratio: Proportion of data for testing
        seed: Random seed for reproducibility
    """
    # Validate ratios
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1"
    
    # Create output directories
    splits = ['train', 'val', 'test']
    for split in splits:
        for dtype in defect_types:
            os.makedirs(os.path.join(output_dir, split, 'images', dtype), exist_ok=True)
            os.makedirs(os.path.join(output_dir, split, 'masks', dtype), exist_ok=True)

    # Process each defect type
    for dtype in tqdm(defect_types, desc="Processing defect types"):
        # Get list of images
        img_dir = os.path.join(input_dir, 'images', dtype)
        mask_dir = os.path.join(input_dir, 'masks', dtype)
        
        images = [f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        np.random.seed(seed)
        np.random.shuffle(images)
        
        # Calculate split indices
        total = len(images)
        train_end = int(train_ratio * total)
        val_end = train_end + int(val_ratio * total)
        
        # Split files
        train_files = images[:train_end]
        val_files = images[train_end:val_end]
        test_files = images[val_end:]
        
        # Copy files to split directories
        def copy_split(files, split):
            for f in tqdm(files, desc=f"Copying {split} files for {dtype}", leave=False):
                # Copy image
                src_img = os.path.join(img_dir, f)
                dst_img = os.path.join(output_dir, split, 'images', dtype, f)
                shutil.copy2(src_img, dst_img)
                
                # Copy mask (assumes same filename with .png extension)
                mask_name = os.path.splitext(f)[0] + '.png'
                src_mask = os.path.join(mask_dir, mask_name)
                if os.path.exists(src_mask):
                    dst_mask = os.path.join(output_dir, split, 'masks', dtype, mask_name)
                    shutil.copy2(src_mask, dst_mask)

        copy_split(train_files, 'train')
        copy_split(val_files, 'val')
        copy_split(test_files, 'test')

# Configuration
DEFECT_TYPES = [
    "Missing_hole",
    "Mouse_bite",
    "Open_circuit",
    "Short",
    "Spur",
    "Spurious_copper"
]

INPUT_DIR = "/kaggle/input/pcb-masks-images/pcb_dataset"
OUTPUT_DIR = "/kaggle/working/"

# Run the splitting
split_dataset(
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    defect_types=DEFECT_TYPES,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    seed=42
)

print("Dataset splitting completed successfully!")

Copying train files for Missing_hole:  98%|█████████▊| 78/80 [00:02<00:00, 32.45it/s]
                                                                                     
Copying val files for Missing_hole:  88%|████████▊ | 15/17 [00:00<00:00, 31.92it/s]
                                                                                   
Copying train files for Mouse_bite:  99%|█████████▉| 79/80 [00:02<00:00, 35.76it/s]
                                                                                   
Copying val files for Mouse_bite:  76%|███████▋  | 13/17 [00:00<00:00, 36.86it/s]
                                                                                 
Copying train files for Open_circuit:  98%|█████████▊| 79/81 [00:02<00:00, 36.41it/s]
                                                                                     
Copying val files for Open_circuit:  94%|█████████▍| 16/17 [00:00<00:00, 35.57it/s]
                                                                        

Dataset splitting completed successfully!
